### **7. WAP to retrain a pretrained imagenet model to classify a medical image dataset.**
---

I am using the [🔗**Brain Tumor MRI Dataset**](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset) , which has the following classes:

- **Glioma Tumor**
- **Meningioma Tumor**
- **Pituitary Tumor**
- **No Tumor**

Here’s a **complete PyTorch program** to retrain a **pretrained ImageNet model** (like `ResNet18`) for this task. It includes:

- Data transforms
- Train/val dataloaders
- Transfer learning setup
- Training + validation loop
- Saving the best model
---

### Folder Structure

```text
Brain_Tumor_Dataset/
├── Training/
│   ├── glioma_tumor/
│   ├── meningioma_tumor/
│   ├── pituitary_tumor/
│   └── no_tumor/
├── Testing/
│   ├── glioma_tumor/
│   ├── meningioma_tumor/
│   ├── pituitary_tumor/
│   └── no_tumor/
```
---

## Retrain ResNet18

In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [8]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# Paths
data_dir = 'Brain_Tumor_Dataset'
train_dir = os.path.join(data_dir, 'Training')
val_dir = os.path.join(data_dir, 'Testing')  # Use 'Testing' as validation

In [10]:
# Hyperparameters
batch_size = 32
learning_rate = 1e-4
num_epochs = 10
num_classes = 4

In [11]:
# Transforms
transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [12]:
# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=transform['val'])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [13]:
# Load pre-trained model
model = models.resnet18(pretrained=True)

C:\Users\harsh\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\harsh\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\harsh/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [03:17<00:00, 237kB/s] 


In [14]:
# Freeze feature extractor
for param in model.parameters():
    param.requires_grad = False

In [15]:
# Replace classifier
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [16]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

In [17]:
# Training + Validation loop
def train_model(model, epochs):
    best_acc = 0.0
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), 'best_brain_tumor_model.pth')

    print(f"\nBest Validation Accuracy: {best_acc:.4f}")

In [18]:
train_model(model, num_epochs)


Epoch 1/10
Train Loss: 1.1471, Acc: 0.5376
Val Loss: 0.9987, Acc: 0.6720

Epoch 2/10
Train Loss: 0.8586, Acc: 0.7400
Val Loss: 0.8040, Acc: 0.7574

Epoch 3/10
Train Loss: 0.7158, Acc: 0.7880
Val Loss: 0.7090, Acc: 0.7864

Epoch 4/10
Train Loss: 0.6293, Acc: 0.8197
Val Loss: 0.6566, Acc: 0.7879

Epoch 5/10
Train Loss: 0.5716, Acc: 0.8256
Val Loss: 0.6141, Acc: 0.8024

Epoch 6/10
Train Loss: 0.5385, Acc: 0.8340
Val Loss: 0.5846, Acc: 0.7956

Epoch 7/10
Train Loss: 0.5123, Acc: 0.8372
Val Loss: 0.5619, Acc: 0.8154

Epoch 8/10
Train Loss: 0.4869, Acc: 0.8477
Val Loss: 0.5412, Acc: 0.8162

Epoch 9/10
Train Loss: 0.4672, Acc: 0.8475
Val Loss: 0.5302, Acc: 0.8192

Epoch 10/10
Train Loss: 0.4503, Acc: 0.8559
Val Loss: 0.5114, Acc: 0.8177

Best Validation Accuracy: 0.8192


---

### To make predictions:

In [22]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import os

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
data_dir = 'Brain_Tumor_Dataset'
val_dir = os.path.join(data_dir, 'Testing')
num_classes = 4

# Define the same transform used in validation
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load validation dataset
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load model
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load('best_brain_tumor_model.pth', map_location=device))
model = model.to(device)
model.eval()

# Evaluate model
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')


C:\Users\harsh\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Test Accuracy: 81.92%


In [ ]:
from PIL import Image

def predict_image(image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform['val'](image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return train_dataset.classes[predicted.item()]


In [21]:
# Example
print(predict_image(r"Brain_Tumor_Dataset\Testing\glioma_tumor\Te-gl_0011.jpg"))

glioma_tumor
